# ATAC-seq
***
<img src="https://www.illumina.com/content/dam/illumina-marketing/images/tools/sequencing/atac-seq.png">
***
<br>**General Workflow**
1. [Trim Adapters](#Trim-Adapters)
2. [Align to genome (BWA)](#Align-to-genome-with-bwa)
3. [Sort and deduplicate reads](#samtools)
    <br>a. [Plot fragment length](#Fragment-Length-Distribution)
4. [Call peaks with Macs2](#Macs2)
    <br>a. [Call peaks with HMMRATAC](#HMMRATAC)


**Analysis Options:**
5. [featureCounts](#featureCounts) 
6. [Motif enrichment](#Homer-motif-analysis)
7. [Genome Tracks](#UCSC-Tracks)
***

## <font color=red>Trim Adapters</font>
> Trim Galore is a wrapper around Cutadapt and FastQC to consistently apply adapter and quality trimming to FastQ files, with extra functionality for RRBS data. [github page](https://github.com/FelixKrueger/TrimGalore)

In [ ]:
#PBS -l nodes=1:ppn=4
#PBS -l walltime=8:00:00

for (( i = 490; i <= 525; i++ ));
do trim_galore --paired --output_dir ~/scratch/Kaech/ATAC/trimmed/\
~/scratch/Kaech/ATAC/fastq/SRR4435${i}_1.fastq.gz \
~/scratch/Kaech/ATAC/fastq/SRR4435${i}_2.fastq.gz; 
done

__Trim galore version 0.6.0 with cutadapt version 2.3__
<br>This version should support multiple cores (10 minute trim to ~ 7 minute trim with 2 cores) but requires Python 3. Additional cores above 2 seem to give no additional benefit.

`--path_to_cutadapt ~/anaconda2_/envs/ATAC/bin/cutadapt` if trim_galore can't find where cutadapt is

In [ ]:
trim_galore --paired -o nextera/ --nextera 1.fastq.gz 2.fastq.gz

## <font color=red>Align to genome with bwa</font>
**This takes a decent amount of time. If possible, use lots of threads**
<br>**Remember to assemble a genome first (mm10.fa)**

bwa mem -t [INT] mm10.fa read1.fq.gz read2.fq.gz > sample.sam
<br>`-t` number of threads

In [ ]:
#PBS -l nodes=1:ppn=8
#PBS -l walltime=12:00:00

module load bwa

for (( i = 490; i <= 525; i++ ));
do bwa mem -t 8 path/to/mm10.fa \
~/scratch/Kaech/ATAC/trimmed/SRR4435${i}_1_val_1.fq.gz \
~/scratch/Kaech/ATAC/trimmed/SRR4435${i}_2_val_2.fq.gz > ~/scratch/Kaech/ATAC/bwa/SRR4435${i}.sam;
done

## <font color=red>samtools</font>
> This formats the aligned reads into a useable format for downstream applications (use version 1.9+, or else markdups will not work)


__`view`: Convert SAM to BAM__ (-b output in BAM; -u output in uncompressed BAM; -h include header in output; -f Only output alignments with all bits set in INT present in the FLAG field; -F Do not output alignments with any bits set in INT present in the FLAG field; -q Skip alignments with MAPQ smaller than INT)
> samtools view [options] in.sam|in.bam|in.cram [region...]

 __`fixmate`: Fill in mate coordinates, ISIZE and mate related flags from a name-sorted alignment.__ (-m adds ms (mate score) tags. These are used by markdup to select the best reads to keep.)
> samtools fixmate [-rpcm] [-O format] in.nameSrt.bam out.bam

 __`sort`: Sort alignments by leftmost coordinates, or by read name when -n is used. An appropriate @HD-SO sort order header tag will be added or an existing one updated if necessary. __(-m Approximately the maximum required memory per thread, specified either in bytes or with a K, M, or G suffix)
> samtools sort [-l level] [-m maxMem] [-o out.bam] [-O format] [-n] [-t tag] [-T tmpprefix] [-@ threads] [in.sam|in.bam|in.cram]

 __`markdup`: Mark duplicate alignments from a coordinate sorted file that has been run through fixmate with the -m option. This program relies on the MC and ms tags that fixmate provides. __(-r removes duplicate reads)
> samtools markdup [-l length] [-r] [-s] [-T] [-S] in.algsort.bam out.bam

In [ ]:
#nodes=1:ppn=8

for (( i = 490; i <= 525; i++));
do samtools view -@ 8 -b -u -h -f 3 -F 256 -F 2048 -q 30 ~/scratch/Kaech/ATAC/bwa/SRR4435${i}.sam \
| samtools fixmate -m - -  \
| samtools sort -m 4G -@ 8 - \
| samtools markdup -r - ~/scratch/Kaech/ATAC/bwa/SRR4435${i}.nodup.bam;
done

## <font color=red>Extract nucleosome-free regions (<100bp)</font>

>This command preserves both ends of reads with fragments < 100bp
    
awk `'$1 ~ /^@/'` finds rows that contain '^@' in the first column    
I think `'$1 ~ /^@/'` preserves the header, since `-h` includes the header in the output. 
<br>`||` is the awk `OR` command; `&&` is the awk `AND` command

In [ ]:
samtools view -h sorted.bam | awk '$9>-100 && $9<100 || $1 ~ /^@/' | samtools view -bS - > sorted.nfr.bam

## <font color=red>Fragment Length Distribution</font>

>[Code Info Here](https://dbrg77.wordpress.com/2017/02/10/atac-seq-insert-size-plotting/)

**samtools view sorted.bam** | #stream reads<br>
**awk '$9>0'** | #output reads with positive fragment length, i.e. left-most reads<br>
**cut -f 9** | #get the fragment length<br>
**sort | uniq -c** | #count the occurrence of each fragment length<br>
**sort -b -k2,2n** | #sort based on fragment length<br>
**sed -e 's/^[ \t]*//'** #remove leading blank of the standard output

`Use R to plot the output .txt`

In [ ]:
samtools index -b sorted.bam

samtools view sorted.bam | awk '$9>0' | cut -f 9 | sort | uniq -c | \
sort -b -k2,2n | sed -e 's/^[ \t]*//' > fragment_length_count.txt

## <font color=red>Macs2</font>

> Call peaks using Macs2

You can call peaks from individual bams, or multiple bam files simultaneously as follows. --nomodel tells macs2 not to treat the sample as ChIP-seq, which requires a specific shifting model.

`Strategy 1:` Call peaks from merged bams from each group. This defeats the purpose of having replicates, because it just merges all the bam files into one.
<br>`Strategy 2:` Call peaks from individual bam files, and then use `bedtools intersect` to find the peaks that are overlapped by each replicate in each group
<br><br> `-q` FDR (default 0.05)

In [ ]:
macs2 callpeak -t bam1 bam2 bam3 -n OutputFolderName --outdir /path/ -g mm --nomodel --shift -100 --extsize 200

In [ ]:
macs2 callpeak -t bam -n OutputFolderName --outdir /path/ -g mm --nomodel --shift -100 --extsize 200

In [ ]:
bedtools intersect -a rep1.narrowPeak -b rep2.narrowPeak -f 0.50 -r \
| bedtools intersect -a - -b rep3.narrowPeak -f 0.50 -r \
| grep "chr" > overlap.narrowPeak

__From Macs2 output, identify:__<br> 1. peaks that are unique to one experimental condition (specific.bed), and take only the first 4 columns of the bed file.
<br>2. "background" peaks from the same experimental condition, and take only the first 4 columns

In [ ]:
bedtools intersect -a experimental_peaks.narrowPeak \
-b comparator_peaks.narrowPeak -v | cut -f 1-4 | grep "chr" \
> experimental_specific.bed
# -v takes the NON-overlap. essentially the same as bedtools subtract

cut -f 1-4 experimental_peaks.narrowPeak | grep "chr" > experimental_background.bed

#### Use the specific and background .bed files as input for [GREAT](http://great.stanford.edu/public/html/)

## <font color=red>HMMRATAC</font>

> Call peaks using [HMMRATAC (Hidden Markov ModeleR)](https://github.com/LiuLabUB/HMMRATAC). Download the software first.



In [ ]:
#First need to make sure the bam only contains paired and successfully pair-matched reads
# -f 3 tells samtools to take the reads that are paired (INT = 1) and matched pairs (INT = 2). INT = 1+2=3
samtools view -b -f 3 sorted.bam > sorted.mapped.bam

#(Optional) For merged replicates, use samtools to merge multiple bams before proceeding to HMMRATAC
samtools merge -n merged.bam sorted.mapped.1.bam...sorted.mapped.n.bam

#Generate a file with genome info (chromosomes and their respective lengths). 
#This can be done using any of the bam files
samtools view -H sorted.mapped.bam | perl -ne 'if(/^@SQ.*?SN:(\w+)\s+LN:(\d+)/){print $1,"\t",$2,"\n"}' > genome.info

#Run HMMRATAC
java -jar HMMRATAC_V1.2.4_exe.jar -b sorted.mapped.bam -i sorted.mapped.bam.bai -g genome.info

## <font color=red>featureCounts</font>

>After calling peaks from all samples, create a reference library of all possible peak regions in the data set. Then assign all of the reads from all .bam files to peaks to obtain read counts. This can be used as input in DESeq2.


In [ ]:
#Create a .saf file (a reference list of all peaks); 
#to be used by featureCounts in lieu of a .gtf reference file
#this file requires a header line (PeakID, Chr, etc.)

cat *peaks.narrowPeak | grep "chr" | sort -k1,1 -k2,2n | bedtools merge -i - | awk \
    'BEGIN{OFS="\t";print "PeakID", "Chr", "Start", "End", "Strand"}\
    {OFS="\t";print "Peak_"NR, $1, $2, $3, "."}' merged.bed > merged.saf

`-a` annotation file
<br>
`-p` count fragments instead of reads. Appropriate for paired-end only

In [ ]:
featureCounts -a merged.saf -F SAF -T 4 -p -o counts_output.txt *.bam

## <font color=red>Homer motif analysis</font>

>[**HOMER**](http://homer.ucsd.edu/homer/index.html) analyzes genomic positions for enriched motifs.  The main idea is that all the user really needs is a file containing genomic coordinates (i.e. a HOMER peak file or  BED file), and HOMER will generally take care of the rest.  To analyze a peak file for motifs, run the following command:

`findMotifsGenome.pl <peak/BED file> <genome> <output directory> -size # [options]`
<br>

To assign peaks to genes:<br>
`annotatePeaks.pl <peak/BED file> <genome> > <output file>`

In [ ]:
#nodes=1:ppn=8

findMotifsGenome.pl experimental_specific.bed mm10 outputDirectoryName -bg \
experimental_background.bed -size 200 -nomotif -bits -mset vertebrates -p 8

In [ ]:
annotatePeaks.pl experimental_specific.bed mm10 | sort -k 2 > experimental.annotated.txt

## <font color=red>UCSC Tracks</font>
>Make browser tracks so you can visualize the data in IGV or UCSC genome browser

 __makeTagDirectory__ basically parses through the alignment file and splits the tags into separate files based on their chromosome.  As a result, several *.tags.tsv files are created in the output directory.  These are made to very efficiently return to the data during downstream analysis.  This also helps speed up the analysis of very large data sets without running out of memory.
<br> __makeUCSCfile__ takes the tag directory and generate the track files for IGV or UCSC genome browser

In [ ]:
for (( i=493; i<=525; i++)); 
do makeTagDirectory ~/scratch/Kaech/ATAC/SRR4435${i} bams/SRR4435${i}.nodup.bam -format bed; 
done

In [ ]:
for dir in SRR4435*;
do makeUCSCfile $dir -o auto;
done